In [1]:
import pandas as pd
from colour import Color

fracsurvivedir = './medianfracsurvivefiles/'
pymoldir = './pymol/'

In [2]:
def MapFracSurvtoPDB(infile, 
                     scriptfile, 
                     colors = ['#fafafa', '#ff0000'], 
                     map_type = 'site_fracsurv', 
                     restrict_to_chain = False, 
                     script_preamble = None):
    '''Writes a colormapping script to be run in pymol; the colormapping is based on fracsurvive 
    to color a structure'''
    df = pd.read_csv(infile)
    df = df.dropna()
    column_names = list(df)
    
    # establish the color spectrum in hex and rgb.
    n_subdivisions = 500 # the color spectrum will be divided into this many discrete colors
    color1 = Color(colors[0])
    color2 = Color(colors[1])
    rgb_spectrum = [c.rgb for c in color1.range_to(color2, n_subdivisions)]
    rgb_spectrum_dict = dict([(i, rgb_spectrum[i]) for i in range(len(rgb_spectrum))])
    
    if map_type == 'site_fracsurv':
        assert 'avgfracsurvive' in column_names
        min_avg = df.min()['avgfracsurvive']  
        max_avg = df.max()['avgfracsurvive']  # the min and max will be mapped to color1 and color2, respectively
        range_avg = max_avg - min_avg
        df['colorindex'] =  (df.avgfracsurvive - min_avg)/range_avg*(n_subdivisions-1)
        
    elif map_type == 'max_fracsurv':
        assert 'maxfracsurvive' in column_names
        min_val = df.min()['maxfracsurvive']  
        max_val = df.max()['maxfracsurvive']  # the min and max will be mapped to color1 and color2, respectively
        range_val = max_val - min_val
        df['colorindex'] =  (df.maxfracsurvive - min_val)/range_val*(n_subdivisions-1)
    
    df['colorindex'] = df['colorindex'].astype(int) # round to nearest index
    df['rgb'] = df['colorindex'].map(rgb_spectrum_dict)        
    site_color_mapping = pd.concat([df['site'], df['rgb']], axis=1)
    
    # write out the script to *scriptfile*:
    f = open(scriptfile, 'w')
    
    if script_preamble:
        preamblef = open(script_preamble, 'r')
        for line in preamblef.readlines():
            f.write(line)
        f.write('\n\n')
        preamblef.close()
    
    for i in range(len(df.index)):
        rgblist = [min(1, c) for c in site_color_mapping.iloc[i]['rgb']]
        r = site_color_mapping.iloc[i]['site']
        if 'HA2' in r:
            f.write("cmd.set_color(\'color{0}_HA2\', \'{1}\')\n".format(r.split(')')[1], rgblist))
            f.write("cmd.color(\'color{0}_HA2\', \'resi {0} and HA2\')\n".format(r.split(')')[1]))
        else:
            f.write("cmd.set_color(\'color{0}\', \'{1}\')\n".format(r, rgblist))
            f.write("cmd.color(\'color{0}\', \'resi {0} and HA1\')\n".format(r))
                    
    f.close()

In [4]:
MapFracSurvtoPDB('{0}/antibody_S139_median_avgsite.csv'.format(fracsurvivedir), 
                 '{0}/S139_median_maxmut.py'.format(pymoldir), 
                 map_type = 'max_fracsurv',
                 script_preamble = './pymol/pymol_preamble.txt')

In [6]:
MapFracSurvtoPDB('{0}/antibody_C179_median_avgsite.csv'.format(fracsurvivedir), 
                 '{0}/C179.py'.format(pymoldir), 
                 map_type = 'max_fracsurv',
                 script_preamble = './pymol/pymol_preamble.txt')

In [9]:
MapFracSurvtoPDB('{0}/antibody_FI6v3_median_avgsite.csv'.format(fracsurvivedir), 
                 '{0}/FI6v3_median_maxmut.py'.format(pymoldir), 
                 map_type = 'max_fracsurv',
                 script_preamble = './pymol/pymol_preamble.txt')

In [10]:
MapFracSurvtoPDB('{0}/antibody_H17L19_median_avgsite.csv'.format(fracsurvivedir), 
                 '{0}/H17L19_median_maxmut.py'.format(pymoldir), 
                 map_type = 'max_fracsurv',
                 script_preamble = './pymol/pymol_preamble.txt')

In [11]:
MapFracSurvtoPDB('{0}/antibody_H17L10_median_avgsite.csv'.format(fracsurvivedir), 
                 '{0}/H17L10_median_maxmut.py'.format(pymoldir), 
                 map_type = 'max_fracsurv',
                 script_preamble = './pymol/pymol_preamble.txt')

In [12]:
MapFracSurvtoPDB('{0}/antibody_H17L7_median_avgsite.csv'.format(fracsurvivedir), 
                 '{0}/H17L7_median_maxmut.py'.format(pymoldir), 
                 map_type = 'max_fracsurv',
                 script_preamble = './pymol/pymol_preamble.txt')

In [9]:
# Make pymol script for FI6v3 max mut to same scale as C179
# In the input file, I changed the last residue of the signal peptide (-1) to have a max mut and avg mut value
# equal to site 38 (H3 numbering) in C179
colors1 = ['#e4dffe', '#1b157a']

MapFracSurvtoPDB('{0}/antibody_FI6v3_median_avgsite_C179scale.csv'.format(fracsurvivedir), 
                 '{0}/FI6v3_median_maxmut_C179scale.py'.format(pymoldir), 
                 map_type = 'max_fracsurv',
                 colors = colors1,
                 script_preamble = './pymol/pymol_preamble.txt')

MapFracSurvtoPDB('{0}/antibody_C179_median_avgsite.csv'.format(fracsurvivedir), 
                 '{0}/C179_maxmut_zoomed.py'.format(pymoldir), 
                 map_type = 'max_fracsurv', 
                 colors = colors1,
                 script_preamble = './pymol/pymol_preamble.txt')

In [47]:
def MapFracSurv3Colors(infile, 
                       scriptfile,  
                       map_type = 'max_fracsurv', 
                       restrict_to_chain = False, 
                       script_preamble = None):
    '''Writes a colormapping script to be run in pymol; the colormapping is based on fracsurvive 
    to color a structure'''
    df = pd.read_csv(infile)
    df = df.dropna()
    column_names = list(df)
    
    # write out the script to *scriptfile*:
    f = open(scriptfile, 'w')
    
    if script_preamble:
        preamblef = open(script_preamble, 'r')
        for line in preamblef.readlines():
            f.write(line)
        f.write('\n\n')
        preamblef.close()
    
    for index,row in df.iterrows():
        r = row['site']
        maxfrac = row['maxfracsurvive']
        if 'HA2' in r:
            s = r.split(')')[1]
            f.write("cmd.alter(\'HA2 and resi {0}\', \'b = {1}\')\n".format(s, maxfrac))
        else:
            f.write("cmd.alter(\'HA1 and resi {0}\', \'b = {1}\')\n".format(r, maxfrac))
            
    f.write("\ncmd.spectrum(\'b\', \'white_orange_hassium\', \'1RVX\')")

    f.close()

In [48]:
MapFracSurv3Colors('{0}/antibody_FI6v3_median_avgsite_C179scale.csv'.format(fracsurvivedir), 
                 '{0}/FI6v3_median_maxmut_C179scale.py'.format(pymoldir),
                 script_preamble = './pymol/pymol_preamble.txt')

MapFracSurv3Colors('{0}/antibody_C179_median_avgsite.csv'.format(fracsurvivedir), 
                 '{0}/C179_maxmut_zoomed.py'.format(pymoldir), 
                 script_preamble = './pymol/pymol_preamble.txt')